In [2]:
import torch
#import clip
from PIL import Image
import os
import numpy as np
import pandas as pd
import json
import cv2
import streamlit as st
import tempfile

In [8]:
path = './PaddleDetection/france_ligue-1/'
tmp = path+os.listdir(path)[0]
tmp

'./PaddleDetection/france_ligue-1/2016-2017_2016-08-12 - 21-00 Bastia 0 - 1 Paris SG_1_720p_5069.jpg'

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open(tmp)).unsqueeze(0).to(device)


with torch.no_grad():
    image_features = model.encode_image(image)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.631   0.247   0.12225]]


In [8]:
type(image_features)

torch.Tensor

In [10]:
clip.available_models()


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [16]:
image_features = image_features.cpu().detach().numpy()

np.save('test_feat.npy', image_features)

In [5]:
len(os.listdir(path))

581318

In [14]:
path2 = './clip_features/'
len(os.listdir(path2))

296294

In [15]:
test_name = sorted(os.listdir(path2))[0]
test_name

'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_1.jpg.npy'

In [16]:
test = np.load(os.path.join(path2, sorted(os.listdir(path2))[0]))

In [17]:
test_name.index('1_720p')

59

In [18]:
test_name[:test_name.index('720p')+4]

'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p'

In [19]:
real = np.load('SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/path/to/SoccerNet/france_ligue-1/2014-2015/2015-04-05 - 22-00 Marseille 2 - 3 Paris SG/1_ResNET_TF2.npy')

In [20]:
combined = np.load(os.path.join(path2, sorted(os.listdir(path2))[0]))
combined = np.vstack((combined, \
                     np.load(os.path.join(path2, sorted(os.listdir(path2))[1]))))

In [21]:
np.load(os.path.join(path2, sorted(os.listdir(path2))[0])).shape

(1, 512)

In [22]:
test_name.index('.jpg.npy')

67

In [23]:
test_name[test_name.index('720p')+5:test_name.index('.jpg.npy')]

'1'

In [24]:
combined = np.vstack((combined, np.load(os.path.join(path2, sorted(os.listdir(path2))[2]))))

In [25]:
combined.shape

(3, 512)

In [26]:
def get_number(str):
    return int(str[str.index('720p')+5:str.index('.jpg.npy')])

list_1 = []
i = 0
sorted_1 = sorted(os.listdir(path2))

while(sorted_1[i][:-10]==sorted_1[0][:-10]):
    list_1.append(sorted_1[i])
    i+=1

sorted_2 = sorted(list_1, key = get_number)

sorted_1[5420:5429]

['feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_992.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_993.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_994.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_995.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_996.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_997.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_998.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_999.jpg.npy',
 'feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_2_720p_1.jpg.npy']

In [146]:
test_name[5:-6][:-9]

'2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1'

In [164]:
def get_number(str):
    return int(str[str.index('720p')+5:str.index('.jpg.npy')])

start = 0
while(start<len(sorted_1)-1):
    i = start
    print(start)
    tmp_list = []
    while(i<len(sorted_1) and sorted_1[i][:sorted_1[i].index('720p')+4]==sorted_1[start][:sorted_1[i].index('720p')+4]):
        tmp_list.append(sorted_1[i])
        i+=1
    sorted_2 = sorted(tmp_list, key = get_number)
    combined = np.load(os.path.join(path2, sorted_1[start]))
    for j in range(1,len(sorted_2)):
        combined = np.vstack((combined, np.load(os.path.join(path2, sorted_2[j]))))
    print(sorted_2[-1])
    np.save("./combined_features/combined_"+sorted_2[-1][5:-9][:-9], combined)
    start += len(sorted_2)

0
feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1_720p_5428.jpg.npy
5428
feat_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_2_720p_6306.jpg.npy
11734
feat_2015-2016_2015-09-26 - 18-30 Nantes 1 - 4 Paris SG_1_720p_5400.jpg.npy
17134
feat_2015-2016_2015-09-26 - 18-30 Nantes 1 - 4 Paris SG_2_720p_5506.jpg.npy
22640
feat_2015-2016_2015-11-07 - 19-00 Paris SG 5 - 0 Toulouse_1_720p_5400.jpg.npy
28040
feat_2015-2016_2015-11-07 - 19-00 Paris SG 5 - 0 Toulouse_2_720p_5460.jpg.npy
33500
feat_2016-2017_2016-08-12 - 21-00 Bastia 0 - 1 Paris SG_1_720p_5400.jpg.npy
38900
feat_2016-2017_2016-08-12 - 21-00 Bastia 0 - 1 Paris SG_2_720p_5762.jpg.npy
44662
feat_2016-2017_2016-09-16 - 21-45 Caen 0 - 6 Paris SG_1_720p_5634.jpg.npy
50296
feat_2016-2017_2016-09-16 - 21-45 Caen 0 - 6 Paris SG_2_720p_5400.jpg.npy
55696
feat_2016-2017_2016-09-23 - 21-45 Toulouse 2 - 0 Paris SG_1_720p_5436.jpg.npy
61132
feat_2016-2017_2016-09-23 - 21-45 Toulouse 2 - 0 Paris SG_2_720p_5400.jpg.npy
66532
feat_

In [93]:
combined.shape

(5428, 512)

In [11]:
np.load('./combined_features/combined_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_2.npy').shape[0]

6306

In [12]:
np.load('SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/path/to/SoccerNet/france_ligue-1/2014-2015/2015-04-05 - 22-00 Marseille 2 - 3 Paris SG/2_ResNET_TF2.npy').shape

(6307, 2048)

In [159]:
len(os.listdir('SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/path/to/SoccerNet/france_ligue-1/2016-2017'))

43

In [3]:
len(os.listdir('combined_features'))

55

In [8]:
path_3 = 'SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/combined_features'
path_4 = 'SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/path/to/SoccerNet/france_ligue-1'
good_list = []
for i in os.listdir(path_3):
    try:
        if np.load(os.path.join(path_3,i)).shape[0] == np.load(os.path.join(path_4, i[9:18], i[19:-6], i[-5]+'_ResNET_TF2.npy')).shape[0]:
            good_list.append(i)
    except IsADirectoryError as error:
        continue

In [9]:
sorted(os.listdir('combined_features'))

['.ipynb_checkpoints',
 'combined_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1.npy',
 'combined_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_2.npy',
 'combined_2015-2016_2015-09-26 - 18-30 Nantes 1 - 4 Paris SG_1.npy',
 'combined_2015-2016_2015-09-26 - 18-30 Nantes 1 - 4 Paris SG_2.npy',
 'combined_2015-2016_2015-11-07 - 19-00 Paris SG 5 - 0 Toulouse_1.npy',
 'combined_2015-2016_2015-11-07 - 19-00 Paris SG 5 - 0 Toulouse_2.npy',
 'combined_2016-2017_2016-08-12 - 21-00 Bastia 0 - 1 Paris SG_1.npy',
 'combined_2016-2017_2016-08-12 - 21-00 Bastia 0 - 1 Paris SG_2.npy',
 'combined_2016-2017_2016-09-16 - 21-45 Caen 0 - 6 Paris SG_1.npy',
 'combined_2016-2017_2016-09-16 - 21-45 Caen 0 - 6 Paris SG_2.npy',
 'combined_2016-2017_2016-09-23 - 21-45 Toulouse 2 - 0 Paris SG_1.npy',
 'combined_2016-2017_2016-09-23 - 21-45 Toulouse 2 - 0 Paris SG_2.npy',
 'combined_2016-2017_2016-10-01 - 18-00 Paris SG 2 - 0 Bordeaux_1.npy',
 'combined_2016-2017_2016-10-01 - 18-00 Paris SG 2 - 0 

In [9]:
good_list.remove('combined_2014-2015_2015-04-05 - 22-00 Marseille 2 - 3 Paris SG_1.npy')
good_list.remove('combined_2016-2017_2017-04-18 - 19-30 Metz 2 - 3 Paris SG_2.npy')
good_list.remove('combined_2016-2017_2017-05-20 - 22-00 Paris SG 1 - 1 Caen_2.npy')

ValueError: list.remove(x): x not in list

In [10]:
len(good_list)

48

In [34]:
for i in os.listdir('combined_features'):
    if i not in good_list and os.path.isfile(os.path.join(path_3,i)):
        os.remove(os.path.join(path_3,i))

In [35]:
len(os.listdir('combined_features'))

49

In [1]:
from SoccerNet.Downloader import getListGames

In [11]:
listGames = getListGames(["train"])

In [12]:
listGames[0]

'england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley'

In [17]:
good_names = []
for i in good_list:
    good_names.append('france_ligue-1/'+i[9:-6].replace('_',listGames[0][11]))

In [28]:
b = good_names[0]
b

'france_ligue-1/2016-2017/2017-02-19 - 23-00 Paris SG 0 - 0 Toulouse'

In [19]:
import json
with open("SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/src/names", "w") as fp:
    json.dump(good_names, fp)
with open("SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/src/names", "r") as fp:
    b = json.load(fp)

In [32]:
tmp = 'combined_'+b[15:].replace('/','_')+'_2.npy'

In [33]:
test_clip = np.load("SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/combined_features/"+tmp)

In [34]:
test_real = np.load("SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/path/to/SoccerNet/france_ligue-1/2016-2017/2017-02-19 - 23-00 Paris SG 0 - 0 Toulouse/2_ResNET_TF2_PCA512.npy")

In [35]:
np.add(test_clip, test_real).shape

(5400, 512)

In [38]:
test_clip = np.concatenate((test_clip, test_real), axis=0)

In [39]:
test_clip.shape

(10800, 512)

In [40]:
path = "SoccerNetv2-DevKit/Task1-ActionSpotting/TemporallyAwarePooling/combined_features"
res = 0
for i in good_list:
    tmp = np.load(os.path.join(path,i))
    if np.isnan(tmp).any():
        print(i)
        res+=1

In [41]:
res

0

In [42]:
import sklearn.metrics
from sklearn.metrics import average_precision_score

In [44]:
average_precision_score(np.array([0, 0, 0, 0, 0]), np.array([0, 0, 0, 0, 0]))

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


-0.0

In [3]:
path = "SoccerNetv2-DevKit/Task1-ActionSpotting/CALF/"

In [4]:
prediction = os.path.join(path, 'inference/outputs/Predictions-v2.json')
file = os.path.join(path,'test_vid_2.mp4')

In [5]:
cap = cv2.VideoCapture(file)

In [6]:
with open(prediction, "r") as f:
    predictions = json.load(f)

In [52]:
cap.get(3)

640.0

In [82]:
int(predictions.get('predictions')[0]['position'])

25500

In [140]:
output_video_path = "output_video.mp4"  # Replace with the desired output video path
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(3)), int(cap.get(4))))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_index = int(cap.get(cv2.CAP_PROP_POS_MSEC))
    labeled_frame = frame.copy()
    
    frame_predictions = predictions.get('predictions')
    
    for i in range(len(frame_predictions)):
        if int(frame_index/1000)*1000==int(int(frame_predictions[i]['position'])/1000)*1000:
            label = frame_predictions[i]["label"]
            confidence = frame_predictions[i]["confidence"]
        #xmin, ymin, xmax, ymax = prediction["bbox"]

        # Draw bounding box on the frame
        #cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
            cv2.putText(labeled_frame, label, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_8)
            cv2.putText(labeled_frame, confidence, (50,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_8)

    out.write(labeled_frame)
    #if cv2.waitKey(1) & 0xFF == ord("q"):
    #    break

cap.release()
out.release()


In [104]:
frame_predictions

[{'gameTime': '1 - 0:25',
  'label': 'Throw-in',
  'position': '25500',
  'half': '1',
  'confidence': '0.6571052670478821'},
 {'gameTime': '1 - 0:31',
  'label': 'Clearance',
  'position': '31000',
  'half': '1',
  'confidence': '0.6067454814910889'},
 {'gameTime': '1 - 0:41',
  'label': 'Shots on target',
  'position': '41000',
  'half': '1',
  'confidence': '0.36006632447242737'},
 {'gameTime': '1 - 0:52',
  'label': 'Shots on target',
  'position': '52000',
  'half': '1',
  'confidence': '0.2590104341506958'},
 {'gameTime': '1 - 1:0',
  'label': 'Ball out of play',
  'position': '60000',
  'half': '1',
  'confidence': '0.7889529466629028'},
 {'gameTime': '1 - 1:6',
  'label': 'Shots on target',
  'position': '66000',
  'half': '1',
  'confidence': '0.550079882144928'},
 {'gameTime': '1 - 1:18',
  'label': 'Shots on target',
  'position': '78000',
  'half': '1',
  'confidence': '0.3410050868988037'},
 {'gameTime': '1 - 1:23',
  'label': 'Goal',
  'position': '83000',
  'half': '1',


In [88]:
predictions.get('predictions')

[{'gameTime': '1 - 0:25',
  'label': 'Throw-in',
  'position': '25500',
  'half': '1',
  'confidence': '0.6571052670478821'},
 {'gameTime': '1 - 0:31',
  'label': 'Clearance',
  'position': '31000',
  'half': '1',
  'confidence': '0.6067454814910889'},
 {'gameTime': '1 - 0:41',
  'label': 'Shots on target',
  'position': '41000',
  'half': '1',
  'confidence': '0.36006632447242737'},
 {'gameTime': '1 - 0:52',
  'label': 'Shots on target',
  'position': '52000',
  'half': '1',
  'confidence': '0.2590104341506958'},
 {'gameTime': '1 - 1:0',
  'label': 'Ball out of play',
  'position': '60000',
  'half': '1',
  'confidence': '0.7889529466629028'},
 {'gameTime': '1 - 1:6',
  'label': 'Shots on target',
  'position': '66000',
  'half': '1',
  'confidence': '0.550079882144928'},
 {'gameTime': '1 - 1:18',
  'label': 'Shots on target',
  'position': '78000',
  'half': '1',
  'confidence': '0.3410050868988037'},
 {'gameTime': '1 - 1:23',
  'label': 'Goal',
  'position': '83000',
  'half': '1',


In [89]:
cap.set(cv2.CAP_PROP_POS_MSEC, 25500)
ret, frame = cap.read()